In [1]:
from sqlalchemy import create_engine
import sqlite3
import pandas as pd

# Create an SQLite database engine
engine = create_engine('sqlite:///ABS_2021_Census.db')

# Load the CSV file
file_path = 'ABS_2021_Census.csv'
data = pd.read_csv(file_path)

# Connect to SQLite database
conn = sqlite3.connect('ABS_2021_Census.db')

# Convert the DataFrame to a SQL table
data.to_sql('ABS_2021_Census', conn, if_exists='replace', index=False)

# Query to fetch the first 5 rows from the table
cursor = conn.cursor()
cursor.execute("SELECT * FROM ABS_2021_Census LIMIT 1")
rows = cursor.fetchall()

# Close the connection
conn.close()

# Display the first few rows
for row in rows:
    print(row)

(1, '101021007', 'Braidwood', 3418.3525, 'https://linked.data.gov.au/dataset/asgsed3/SA2/101021007', 12.0, 12.0, 26.0, 9.0, 6.0, 18.0, 14.0, 21.0, 33.0, 6.0, 49.0, 54.0, 22.0, 91.0, 113.0, 21.0, 59.0, 79.0, 69.0, 50.0, 120.0, 75.0, 55.0, 128.0, 70.0, 54.0, 129.0, 105.0, 36.0, 136.0, 79.0, 24.0, 105.0, 66.0, 15.0, 82.0, 139.0, 34.0, 176.0, 84.0, 0.0, 84.0, 93.0, 11.0, 104.0, 45.0, 0.0, 48.0, 150.0, 16.0, 166.0, 90.0, 4.0, 95.0, 18.0, 16.0, 30.0, 1160.0, 559.0, 1719.0, '01d66027-2c46-4b72-9162-2abc87ca39e3', 0.339397031774297, 3.9136945523189)


In [2]:
pip install langchain==0.1.12

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.agents import create_sql_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.prompts import ChatPromptTemplate

llm = OpenAI()

# Create a connection to your SQLite database
db = SQLDatabase.from_uri('sqlite:///digital_atlas_of_australia.db')

# Initialize the toolkit for SQL interaction
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Fetch database schema (optional but can help LLM understand the structure)
database_schema = db.get_table_info()

# Custom function to generate SQL query without executing it
def generate_sql_query(input_text):
    # Use the ChatPromptTemplate to create a prompt for generating SQL queries
    query_prompt_template = ChatPromptTemplate.from_template(
        "Given the following database schema:\n{database_schema}\nGenerate a SQL query for the following task: {input_text}, but do not execute it."
    )
    
    # Combine the toolkit and LLM to create the final prompt and execute it to get the query
    prompt = query_prompt_template.format(database_schema=database_schema, input_text=input_text)
    
    # Use the toolkit LLM to get the SQL query
    sql_query = toolkit.llm(prompt)
    
    return sql_query

# Example usage
input_text = input("Hello, how may I help: ")
input_text = input_text + "(from atlas_table using tag field and return all records"
sql_query = generate_sql_query(input_text)
print(f"Generated SQL Query: {sql_query}")

Hello, how may I help:  give me rent data


Generated SQL Query: 

SELECT * FROM atlas_data WHERE tags LIKE '%Rent%'


In [4]:
# Create a connection to your SQLite database
db = SQLDatabase.from_uri('sqlite:///ABS_2021_Census.db')

# Initialize the toolkit for SQL interaction
toolkit = SQLDatabaseToolkit(from langchain.agents import create_sql_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.prompts import ChatPromptTemplate
import os

# Set your OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI LLM with the API key
llm = OpenAI(openai_api_key=OPENAI_API_KEY)db=db, llm=llm)

# Fetch database schema (optional but can help LLM understand the structure)
database_schema = db.get_table_info()

# Custom function to generate SQL query without executing it
def generate_sql_query(input_text):
    # Use the ChatPromptTemplate to create a prompt for generating SQL queries
    query_prompt_template = ChatPromptTemplate.from_template(
        "Given the following database schema:\n{database_schema}\nGenerate a SQL query for the following task: {input_text}, but do not execute it."
    )
    
    # Combine the toolkit and LLM to create the final prompt and execute it to get the query
    prompt = query_prompt_template.format(database_schema=database_schema, input_text=input_text)
    
    # Use the toolkit LLM to get the SQL query
    sql_query = toolkit.llm(prompt)
    
    return sql_query

# Example usage
input_text = input("Hello, how may I help: ")
# Extract only the digits and convert them to an integer
parsed_int = int(''.join(filter(str.isdigit, input_text)))
if parsed_int == 1000:
    input_text = input_text + "and $2000 and $3000 and $4000"
elif parsed_int == 2000:
    input_text = input_text + "$3000 and $4000"
elif parsed_int == 3000:
    input_text = input_text + "$4000"
input_text = input_text + "(from ABS_2021_Census table using columns as description, select that column and return all records"
sql_query = generate_sql_query(input_text)
print(f"Generated SQL Query: {sql_query}")

Hello, how may I help:  show me households which earn more than $2000


Generated SQL Query: 

SELECT "SA2_Name", "Household_Income:_$2000-$2499:_Total", "Household_Income:_$2500-$2999:_Total", "Household_Income:_$3000-$3499:_Total", "Household_Income:_$3500-$3999:_Total", "Household_Income:_$4000_or_more:_Total" 
FROM ABS_2021_Census
WHERE "Household_Income:_$2000-$2499:_Total" > 0 OR "Household_Income:_$2500-$2999:_Total" > 0 OR "Household_Income:_$3000-$3499:_Total" > 0 OR "Household_Income:_$3500-$3999:_Total" > 0 OR "Household_Income:_$4000_or_more:_Total" > 0;
